# AWS - RDS MySQL
### Paquetes requeridos
Librería de Python necesaria para establecer conexión a una base de datos MySQL

In [259]:
import pymysql
import sqlalchemy
import pandas as pd


### Credenciales
Para acceder a la BD necesitaremos tres datos:
1. Endpoint/host: la dirección donde se encuentra el servidor
2. Usuario
3. Contraseña

In [260]:
host = "popcorn-db.cb4gusk0kj8q.eu-central-1.rds.amazonaws.com"
username = "admin"
password = "jzd13051998"
port = 3306
name = 'popcorn-db'

### Conexion BD

---

In [261]:
'''
pymysql.cursors.DictCursor para que los resultados que devuelva sean diccionarios,
por defecto devuelve tuplas. Asi podemos acceder por clave a las columnas.
'''

db = pymysql.connect(host = host,
                     user = username,
                     password = password,
                     cursorclass = pymysql.cursors.DictCursor
)

# El objeto cursor es el que ejecutará las queries y devolverá los resultados

cursor = db.cursor()

### Version DB

In [262]:
cursor.execute('SELECT VERSION()')
version = cursor.fetchone()
print(f'MySQL version: {version}')

MySQL version: {'VERSION()': '8.0.40'}


### Creación de DB

In [263]:
# Verificar si la base de datos existe antes de crearla
cursor.execute("SHOW DATABASES;")  # Ejecuta la consulta
databases = [db["Database"] for db in cursor.fetchall()]  # Obtiene las bases de datos

if "popcorn_db" not in databases:  # Si no está en la lista, la creamos
    create_db = "CREATE DATABASE popcorn_db;"
    cursor.execute(create_db)
    print("Base de datos creada correctamente.")
else:
    print("La base de datos ya existe, no es necesario crearla.")

    

La base de datos ya existe, no es necesario crearla.


In [264]:
cursor.execute('SHOW DATABASES')
cursor.fetchall()

[{'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'},
 {'Database': 'popcorn_db'},
 {'Database': 'sys'}]

### Creación de tablas

In [265]:
use_db='''USE popcorn_db'''
cursor.execute(use_db)

0

In [266]:
# Crear una tabla

# Verificar si la tabla 'interactions' existe antes de crearla
cursor.execute("SHOW TABLES;")  # Ejecuta la consulta para listar las tablas existentes
tables = [table[f"Tables_in_popcorn_db"] for table in cursor.fetchall()]  # Obtiene la lista de tablas

if "interactions" not in tables:  # Si la tabla no existe, la creamos
    create_table = '''
    CREATE TABLE interactions (
        id INT NOT NULL AUTO_INCREMENT,
        pregunta TEXT,
        respuesta TEXT,
        PRIMARY KEY (id)
    );
    '''
    cursor.execute(create_table)
    print("✅ Tabla 'interactions' creada correctamente.")
else:
    print("La tabla 'interactions' ya existe, no es necesario crearla.")


La tabla 'interactions' ya existe, no es necesario crearla.


### Seleccionar la BD

In [267]:
# checkear todas las tablas que tiene mi db
cursor.execute('SHOW TABLES')
cursor.fetchall()

[{'Tables_in_popcorn_db': 'interactions'}]

In [268]:
db.ping(reconnect=True)
cursor.execute('SELECT * FROM interactions')
cursor.fetchall()

()

In [269]:
db.ping(reconnect=True)
cursor.execute('DELETE FROM interactions')
db.commit()
cursor.fetchall()

[]

In [270]:
db.ping(reconnect=True)
cursor.execute('SHOW ENGINE INNODB STATUS;')
db.commit()
cursor.fetchall()

[{'Type': 'InnoDB',
  'Name': '',
  'Status': "\n=====================================\n2025-03-02 20:13:47 70369790309712 INNODB MONITOR OUTPUT\n=====================================\nPer second averages calculated from the last 34 seconds\n-----------------\nBACKGROUND THREAD\n-----------------\nsrv_master_thread loops: 1 srv_active, 0 srv_shutdown, 78 srv_idle\nsrv_master_thread log flush and writes: 0\n----------\nSEMAPHORES\n----------\nOS WAIT ARRAY INFO: reservation count 2\nOS WAIT ARRAY INFO: signal count 3\nRW-shared spins 0, rounds 0, OS waits 0\nRW-excl spins 0, rounds 0, OS waits 0\nRW-sx spins 0, rounds 0, OS waits 0\nSpin rounds per wait: 0.00 RW-shared, 0.00 RW-excl, 0.00 RW-sx\n------------\nTRANSACTIONS\n------------\nTrx id counter 6419\nPurge done for trx's n:o < 6411 undo n:o < 0 state: running but idle\nHistory list length 5\nLIST OF TRANSACTIONS FOR EACH SESSION:\n---TRANSACTION 351844750596208, not started\n0 lock struct(s), heap size 1128, 0 row lock(s)\n---TRA

In [271]:
db.ping(reconnect=True)
cursor.execute('ROLLBACK')
db.commit()
cursor.fetchall()

cursor.execute('SET GLOBAL innodb_lock_wait_timeout = 1')
db.commit()
cursor.fetchall()


OperationalError: (1227, 'Access denied; you need (at least one of) the SUPER or SYSTEM_VARIABLES_ADMIN privilege(s) for this operation')

In [272]:
cursor.execute('SHOW PROCESSLIST')
processes = cursor.fetchall()
for process in processes:
    process_id = process["Id"]
    user = process["User"]
    if user in ["rdsadmin", "event_scheduler"]:
        print(f"⚠️ No se puede terminar {process_id}: Es un proceso del sistema ({user}).")
        continue

    try:
        cursor.execute(f"KILL {process_id}")
        db.commit()
        print(f"✅ Proceso {process_id} terminado.")
    except Exception as e:
        print(f"⚠️ No se pudo terminar {process_id}: {e}")


⚠️ No se puede terminar 5: Es un proceso del sistema (event_scheduler).
⚠️ No se puede terminar 8: Es un proceso del sistema (rdsadmin).
⚠️ No se pudo terminar 9: (1317, 'Query execution was interrupted')


In [275]:
db.ping(reconnect=True)
cursor.execute("SELECT * FROM information_schema.innodb_trx WHERE trx_state = 'LOCK WAIT'")
cursor.fetchall()
db.commit()

In [276]:
processes = cursor.fetchall()
for process in processes:
    process_id = process["Id"]
    user = process["User"]
    try:
        cursor.execute(f"KILL {process_id}")
        db.commit()
        print(f"✅ Proceso {process_id} terminado.")
    except Exception as e:
            print(f"⚠️ No se pudo terminar {process_id}: {e}")



In [40]:
# drop_table = '''DROP TABLE country'''
# cursor.execute(drop_table)

In [123]:
# Crear conexión con pymysql
try:
    connection = pymysql.connect(
        host=host,
        user=username,
        password=password,
        database=name,
        cursorclass=pymysql.cursors.DictCursor
    )
    print("✅ Conexión exitosa a MySQL en AWS")
except Exception as e:
    print(f"❌ Error al conectar con MySQL: {e}")

❌ Error al conectar con MySQL: (1049, "Unknown database 'popcorn-db'")


In [ ]:
# import psycopg2
# from psycopg2.extras import DictCursor

# # Parámetros de conexión
# host = "database-1.crerrxwepn0b.us-east-2.rds.amazonaws.com"
# username = "postgres"
# password = "12345678"


# # Conexión a PostgreSQL

# conn = psycopg2.connect(
#     host=host,
#     user=username,
#     password=password,
#     cursor_factory=DictCursor  # Devuelve filas como diccionarios
# )
# print("Conexión exitosa a PostgreSQL")

# # Crear un cursor
# cursor = conn.cursor()

### Insertar datos

In [41]:
# insert_data = '''
# INSERT INTO country (name, continent, population, gdp)
# VALUES ('Spain', 'Europe', 47, 1.74)
# '''
# cursor.execute(insert_data)

---

In [6]:
'''
pymysql.cursors.DictCursor para que los resultados que devuelva sean diccionarios,
por defecto devuelve tuplas. Asi podemos acceder por clave a las columnas.
'''

db = pymysql.connect(host = host,
                     user = username,
                     password = password,
                     cursorclass = pymysql.cursors.DictCursor
)

# El objeto cursor es el que ejecutará las queries y devolverá los resultados

cursor = db.cursor()

In [ ]:
# import psycopg2
# from psycopg2.extras import DictCursor

# # Parámetros de conexión
# host = "database-1.crerrxwepn0b.us-east-2.rds.amazonaws.com"
# username = "postgres"
# password = "12345678"


# # Conexión a PostgreSQL

# conn = psycopg2.connect(
#     host=host,
#     user=username,
#     password=password,
#     cursor_factory=DictCursor  # Devuelve filas como diccionarios
# )
# print("Conexión exitosa a PostgreSQL")

# # Crear un cursor
# cursor = conn.cursor()

### Version DB

In [4]:
'''
Este es el engine version de la BD de AWS
fechtone trae la primera linea de la consulta
El execute() devuelve el numero de filas a las que ha afectado la query,
en este caso, devuelve una unica fila.

Execute se guarda en la conexion pero hasta que no hacemos commit
no se ejecutan las queries
de insert de datos y esas cosas...
'''

cursor.execute('SELECT VERSION()')
version = cursor.fetchone()
print(f'MySQL version: {version}')

MySQL version: {'VERSION()': '8.0.39'}


### Creación de DB

In [6]:
cursor.execute('SHOW DATABASES')
cursor.fetchall()

[{'Database': 'country_database'},
 {'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'},
 {'Database': 'sys'}]

### Creación de tablas

In [7]:
'''
Creamos una tabla
Esto da error porque no le hemos dicho al servidor qué BD
queremos usar

'''

# crear una tabla que se llame country

create_table = '''
CREATE TABLE country (
    id INT NOT NULL auto_increment,
    name TEXT,
    continent TEXT,
    population INT,
    gdp DOUBLE,
    primary key (id)
)
'''
cursor.execute(create_table)

OperationalError: (1046, 'No database selected')

### Seleccionar la BD

In [8]:
# Para usar la BD  recien creada

cursor.connection.commit()
use_db = ''' USE country_database'''
cursor.execute(use_db)

0

In [9]:
# checkear todas las tablas que tiene mi db
cursor.execute('SHOW TABLES')
cursor.fetchall()

()

In [11]:

# drop_table = '''DROP TABLE country'''
# cursor.execute(drop_table)


In [10]:
'''
Creamos una tabla
Esto da error porque no le hemos dicho al servidor qué BD
queremos usar

'''

# crear una tabla que se llame country

create_table = '''
CREATE TABLE country (
    id INT NOT NULL auto_increment,
    name TEXT,
    continent TEXT,
    population INT,
    gdp DOUBLE,
    primary key (id)
)
'''
cursor.execute(create_table)

0

### Insertar datos

In [11]:
insert_data = '''
INSERT INTO country (name, continent, population, gdp)
VALUES ('Spain', 'Europe', 47, 1.74)
'''
cursor.execute(insert_data)

1

In [12]:
name = 'France'
continent = 'Europe'
population = 54
gdp = 2

In [13]:
insert_data = '''
INSERT INTO country (name, continent, population, gdp)
VALUES ('%s', '%s', '%s', '%s')
''' % (name,continent,population,gdp)

cursor.execute(insert_data)

1

### Leer datos

In [14]:
sql = '''SELECT * FROM country'''
cursor.execute(sql)

2

In [15]:
mi_lista = cursor.fetchall()
len(mi_lista)

2

In [16]:
mi_lista

[{'id': 1,
  'name': 'Spain',
  'continent': 'Europe',
  'population': 47,
  'gdp': 1.74},
 {'id': 2,
  'name': 'France',
  'continent': 'Europe',
  'population': 54,
  'gdp': 2.0}]

In [17]:
import pandas as pd

In [18]:
pd.DataFrame(mi_lista)

,id,name,continent,population,gdp
0,1,Spain,Europe,47,1.74
1,2,France,Europe,54,2.00


### Guardar los cambios
Hay que ejecutar el commit antes de cerrar la sesión de la BD para que se guarden todos los cambios

In [19]:
db.commit()

### Excepciones
Se recomienda rodear este tipo de sentencias con un try/except, ya que suele ser software productivo y tiene que saber manejar fallos

In [20]:
sql = '''SELECT * FROM countryy'''
cursor.execute(sql)
print(cursor.fetchall())
print('Proceso finalizado')

ProgrammingError: (1146, "Table 'country_database.countryy' doesn't exist")

In [21]:
try:
    sql = '''SELECT * FROM countryy'''
    cursor.execute(sql)
    print(cursor.fetchall())
except Exception as e:
    print(e)

print('Proceso finalizado')

(1146, "Table 'country_database.countryy' doesn't exist")
Proceso finalizado


### Insertar datos de un CSV

In [26]:
# # leemos los datos para comprobar que se han ingestado correctamente
# sql = '''SELECT * FROM country'''
# cursor.execute(sql)
# mi_tabla = cursor.fetchall()
# mi_tabla

### Cerrar la conexión cuando acabemos

In [73]:
# Cerrar conexión mysql
db.close()

In [8]:
# Cerrar conexión sqlalchemy
engine.dispose()

NameError: name 'engine' is not defined